In [1]:
import pandas as pd
import geopandas as gpd
from IPython.display import display
import requests as re
from io import BytesIO
from os import path


from core.downloads.geosampa import get_capabilities, get_features

# GT Saúde

O Grupo de Trabalho de Saúde definiu os indicadores sem a utilização dos formulários, em reuniões posteriores. Por isso, os indicadores serão descritos diretamente neste notebook para documentação.

A base inicial para os indicadores é o capítulo 7, Eficiência nas macrorregiões de saúde no Sistema Único de Saúde: uma abordagem comparativa – 2008-2017, do livro [SUS: avaliação da eficiência do gasto público em saúde](https://repositorio.ipea.gov.br/handle/11058/12029). Este capítulo busca associar os custos e infraestrutura relacionada ao SUS e o impacto na saúde da população de cada macrorregião, utilizando as seguintes variáveis:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Despesas per capita em atenção básica e vigilância;
- Despesas per capita em atenção especializada e Suporte profilático terapêutico;
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Equipamentos respiradores e ventiladores SUS/hab. (x100.000);
- População (em mil hab.);
- Área (km2);
- Densidade demográfica (hab./km²);
- Idosos %;
- PIB per capita;
- Usuários de planos de saúde %.


A partir dessas referência, o GT avaliou a disponibilidade de dados semelhantes a nível intramunicipal e complementou com informações de produção dos serviços de saúde, chegando na seguinte lista de variáveis, sempre que possível agregando por Subprefeitura:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Leitos de UTI/hab. (x100.000);
- Despesas liquidadas em atenção básica e vigilância;
- Despesas liquidadas em atenção especializada e Suporte profilático terapêutico;
- Consultas na Atenção Básica;
- Consultas na Atenção Especializada;
- Consultas na Urgência/Emergência;
- Número de Unidades Básicas de Saúde;
- Número de Ambulatórios especializados;
- Número de Unidades de Saúde Mental;
- Número de Unidades de Urgência e Emergência;
- Número de Hospitais;
- Número de Unidades DST/AIDS;
- Número de demais unidades;

A partir dessa definição, vamos obter as variáveis.

# Extração das variáveis e transformação inicial

## Número de unidades

Estabelecimentos de saúde municipais, estaduais, federais e privados do município estão disponíveis no GeoSampa, em diferentes camadas. Primeiro, vamos avaliar quais são essas camadas e, depois, baixá-las e agrupá-las da maneira decidida pelo GT.

In [ ]:
equipamento_saude_typenames = get_capabilities('equipamento_saude')
equipamento_saude_typenames

Primeiro, removemos todas as camadas que não representam os pontos das unidades de saúde. Além disso, a camada de vigilância não retorna nenhuma unidade, então vamos removê-la também.

In [ ]:
del equipamento_saude_typenames[0]
del equipamento_saude_typenames[1]
del equipamento_saude_typenames[4]
del equipamento_saude_typenames[8]
del equipamento_saude_typenames[7]

equipamento_saude_typenames

In [4]:
gdfs_equip = []

for t in equipamento_saude_typenames:
    gdfs_equip.append(get_features(t.get('name')))

In [ ]:
for gdf in gdfs_equip:
    display(gdf.crs)
    display(gdf.head(2))

In [ ]:
gdf_equip = pd.concat(gdfs_equip)
gdf_equip = gpd.GeoDataFrame(gdf_equip)
gdf_equip = gdf_equip.set_geometry(gdfs_equip[0].geometry.name)
gdf_equip = gdf_equip.set_crs(gdfs_equip[0].crs)
gdf_equip

## Consultas médicas

Os dados de consultas médicas serão obtidos do boletim "Saúde em dados", disponibilizado anualmente pela CEINFO, da Secretaria Municipal de Saúde. Existe uma versão do boletim em excel, regionalizado por subprefeituras. Primeiro, vamos confirmar as planilhas disponíveis no arquivo.

In [ ]:
url = 'https://capital.sp.gov.br/documents/d/saude/tabelas_ceinfo_dados_sub_2024_v3_rev09012025'

response = re.get(url)
response.raise_for_status()
xlsx_file = BytesIO(response.content)
excel = pd.ExcelFile(xlsx_file)

excel.sheet_names

A tabela desejada é a `Consultas Medicas_Odontológicas`, então vamos abri-la como um dataframe.

In [ ]:
sheet_name = 'Consultas Medicas_Odontológicas'
header = [0, 1]
skiprows = 5
df_consultas = pd.read_excel(url,
                   sheet_name=sheet_name,
                   skiprows=skiprows,
                   header=header,
                   thousands='.')
df_consultas

Como a tabela original vem em um formato amigável para humanos, precisamos ajustá-la para melhorar a sua utilização computacional. A primeira coisa é filtrar linhas e colunas carregadas por engano, por conter informações de fonte ou caracteres não imprimíveis.

In [ ]:
df_consultas = df_consultas.iloc[:32, 1:]

df_consultas

Depois, precisamos remover as colunas de totais, que resultariam em duplicação dos valores caso fossem mantidas.

In [ ]:
cols_total_level0 = df_consultas.columns.get_level_values(0).str.contains('total', case=False)
cols_total_level1 = df_consultas.columns.get_level_values(1).str.contains('total', case=False)

df_consultas = df_consultas.loc[:, ~cols_total_level0 & ~cols_total_level1]

df_consultas

Agora, vamos transformar os níveis das colunas em novas colunas de valores (despivotar a tabela).

In [ ]:
id_col = df_consultas.columns[0]
df_consultas_ajustado = pd.melt(df_consultas, id_vars=[id_col])
df_consultas_ajustado

Agora, renomeamos as colunas.

In [ ]:
df_consultas_ajustado.columns = ['Subprefeitura', 'Categoria', 'Subcategoria', 'Qtd_Consultas']
df_consultas_ajustado

Em seguida, removemos as linhas de consultas odontológicas.

In [ ]:
df_consultas_ajustado = df_consultas_ajustado[~df_consultas_ajustado['Categoria'].str.contains('Odontológica')]
df_consultas_ajustado

Ainda, substituímos os `-` por 0 na coluna de quantidade de consultas.

In [ ]:
hifen = df_consultas_ajustado['Qtd_Consultas'] == '-'
df_consultas_ajustado.loc[hifen, 'Qtd_Consultas'] = 0
df_consultas_ajustado.loc[:, 'Qtd_Consultas'] = df_consultas_ajustado.loc[:, 'Qtd_Consultas'].astype(int)
df_consultas_ajustado

Por último, vamos corrigir os nomes das categorias.

In [ ]:
df_consultas_ajustado['Categoria'].value_counts()

In [ ]:
df_consultas_ajustado.loc[:, 'Categoria'] = df_consultas_ajustado.loc[:, 'Categoria'].apply(lambda c: 'Consulta Médica na Atenção Básica' if 'Atenção Básica' in c else c)

df_consultas_ajustado.loc[:, 'Categoria'] = df_consultas_ajustado.loc[:, 'Categoria'].apply(lambda c: 'Consulta Médica/Atendimento em Urgência/Emergência' if 'Urgência' in c else c)

df_consultas_ajustado

## Leitos de internação e complementares

Os dados de leitos não estão disponíveis de maneira regionalizada no boletim "Saúde em dados", então terão de ser obtidos de outra fonte. Eles estão disponíveis na plataforma [TabNet](http://tabnet.saude.prefeitura.sp.gov.br/cgi/deftohtm3.exe?secretarias/saude/TABNET/cnes/leito.def). Entretanto, os links dos arquivos gerados nessa plataforma podem não ser permanentes, então utilizaremos uma versão já baixada do arquivo csv.

Obs: caso a necessidade fosse mais geral ou a atualização mais frequente, seria possível desenvolver um *scrapper* para a plataforma, mas estrapola o escopo deste trabalho.

In [ ]:
data_path = path.join('dados', 'saude')
leitos_filename = 'A161721192_29_142_217.csv'
leitos_file_path = path.join(data_path, leitos_filename)

df_leito = pd.read_csv(leitos_file_path,
                        skiprows = 3,
                        sep = ';',
                        encoding = 'latin1')
df_leito

Primeiro, vamos excluir as linhas sobressalentes ao final do dataframe e a coluna de totalização.

In [ ]:
df_leito = df_leito.iloc[:32]
df_leito

In [ ]:
cols_total = df_leito.columns.str.lower().str.contains('total')
df_leito = df_leito.loc[:,~cols_total]
df_leito

Em seguida, "despivotamos" a tabela.

In [ ]:
df_leito_ajustado = df_leito.melt(id_vars=['Pref Regional'],
                                    var_name='Categoria específica',
                                    value_name='Qtd leitos')
df_leito_ajustado

Por último, substituimos os `-` por 0.

In [ ]:
hifen = df_leito_ajustado['Qtd leitos'].str.contains('-')
df_leito_ajustado.loc[hifen,'Qtd leitos'] = 0
df_leito_ajustado.loc[:,'Qtd leitos'] = df_leito_ajustado.loc[:,'Qtd leitos'].astype(int)
df_leito_ajustado

Posteriormente, cada categoria específica dos leitos será categorizada em Leitos de internações, leitos complementares ou leitos de UTI, de acordo com as variáveis definidas.

## Mortalidade

Os dados sobre mortalidade também serão extraídos da plataforma [TabNet](http://tabnet.saude.prefeitura.sp.gov.br/cgi/deftohtm3.exe?secretarias/saude/TABNET/cnes/leito.def). Novamente, como os links dos arquivos gerados nessa plataforma podem não ser permanentes, utilizaremos uma versão já baixada do arquivo csv.

In [ ]:
data_path = path.join('dados', 'saude')
mortalidade_filename = 'A180859192_29_141_183.csv'
mortalidade_file_path = path.join(data_path, mortalidade_filename)

df_mortalidade = pd.read_csv(mortalidade_file_path,
                        skiprows = 3,
                        sep = ';',
                        encoding = 'latin1')
df_mortalidade

Primeiro, vamos excluir as linhas sobressalentes ao final do dataframe e a coluna de totalização.

In [ ]:
df_mortalidade = df_mortalidade.iloc[:32]
df_mortalidade

In [ ]:
cols_total = df_mortalidade.columns.str.lower().str.contains('total')
df_mortalidade = df_mortalidade.loc[:,~cols_total]
df_mortalidade

Em seguida, "despivotamos" a tabela.

In [ ]:
df_mortalidade_ajustado = df_mortalidade.melt(id_vars=['Subprefeitura residência'],
                                    var_name='Causa específica',
                                    value_name='Qtd óbitos')
df_mortalidade_ajustado

Por último, substituimos os `-` por 0.

In [ ]:
hifen = df_mortalidade_ajustado['Qtd óbitos'].astype(str).str.contains('-')
df_mortalidade_ajustado.loc[hifen,'Qtd óbitos'] = 0
df_mortalidade_ajustado.loc[:,'Qtd óbitos'] = df_mortalidade_ajustado.loc[:,'Qtd óbitos'].astype(int)
df_mortalidade_ajustado

Posteriormente, cada causa específica dos óbitos será categorizado em Evitável, Tratável ou Evitável/Tratável, de acordo com uma tabela auxiliar definida pelo GT.

## Despesas

Para que seja possível relacionar as despesas com os outros indicadores, podemos usar os dados de regionalização do orçamento, disponíveis de acordo com o valor de detalhamento da ação.

Esses dados ficam disponíveis na [página de prestação de contas públicas da Secretaria Municipal da Fazenda](https://orcamento.sf.prefeitura.sp.gov.br/orcamento/execucao.php), na Base de Dados da Regionalização da Execução Orçamentária.

In [ ]:
despesas_url = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2023/basedadosDA_1223.csv'
df_despesa = pd.read_csv(despesas_url,
                         encoding='latin1',
                         sep=';',
                         decimal=',')

df_despesa

Primeiro, vamos filtrar os dados para manter apenas a função Saúde do orçamento.

In [ ]:
df_despesa = df_despesa[df_despesa['DESCRIÇÃO_FUNÇÃO']=='Saúde']
df_despesa

In [ ]:
group_cols = ['CÓDIGO_SUBFUNÇÃO',
        'DESCRIÇÃO_SUBFUNÇÃO',
        'CÓDIGO_PROGRAMA',
        'DESCRIÇÃO_PROGRAMA',
        'REGIÃO',
        'SUBPREFEITURA',
        'DISTRITO',
        'TIPO_REGIONALIZAÇÃO']

metric_cols = ['VALOR_DETALHAMENTO_AÇÃO']

df_despesa_agrupado = df_despesa.groupby(group_cols).sum().reset_index().loc[:, group_cols + metric_cols]
df_despesa_agrupado

Vamos ver quais valores aparecem como subfunção.

In [ ]:
df_despesa_agrupado['DESCRIÇÃO_SUBFUNÇÃO'].value_counts()

Para os indicadores de "Despesas liquidadas em atenção básica e vigilância" e "Despesas liquidadas em atenção especializada e Suporte profilático terapêutico", parece que as subfunções serão o suficiente, com algumas subfunções não se encaixando em nenhuma das categorias.

Baixados todos os dados, vamos começar as transformações para adequar as categorias dos dados às necessidades dos indicadores.